Data 102 Final Project - US Airbnb Listing Price Recommender




In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 14.4 MB/s 
     |████████████████████████████████| 6.3 MB 57.3 MB/s 
     |████████████████████████████████| 15.4 MB 53.9 MB/s 


In [ ]:
!pip install folium

In [ ]:
import geopandas as gpd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)
sns.set_palette('Reds') 

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.model_selection import RandomizedSearchCV

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
airbnb_json = pd.read_json("airbnb-listings.json", orient = 'records')
airbnb_json.head()

ValueError: ignored

In [ ]:
airbnb = pd.json_normalize(airbnb_json.fields)
airbnb.head()

In [ ]:
airbnb.shape

In [ ]:
airbnb.columns

In [ ]:
info = ['id', 'name', 'host_id', 'host_name', 'host_since', 'host_total_listings_count', 'neighbourhood_cleansed', 'city', 'state', 'market', 'country_code', 'latitude',
'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'weekly_price', 'monthly_price','minimum_nights', 'maximum_nights', 'number_of_reviews',
'review_scores_rating', 'reviews_per_month', 'cancellation_policy']

In [ ]:
filtered_airbnb = airbnb[info]
filtered_airbnb.head()

In [ ]:
filtered_airbnb.isna().sum()

In [ ]:
##sophie

In [ ]:
filtered_airbnb['amenities'] = filtered_airbnb['amenities'].str.split(',')
filtered_airbnb = filtered_airbnb[filtered_airbnb["amenities"].notna()]

# amenity_list = filtered_airbnb['amenities'].explode().unique()

# for newcol in amenity_list:
#     filtered_airbnb[newcol]= filtered_airbnb['amenities'].apply(lambda x: any([newcol in x])).astype(int)
# filtered_airbnb

In [ ]:
filtered_airbnb['state'] = filtered_airbnb['state'].str.upper()

In [ ]:
filtered_airbnb.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
#Jenna

In [ ]:
filtered_airbnb["review_scores_rating"] = filtered_airbnb["review_scores_rating"].fillna(filtered_airbnb.groupby("host_id")["review_scores_rating"].transform("mean"))
filtered_airbnb["reviews_per_month"] = filtered_airbnb["reviews_per_month"].fillna(filtered_airbnb.groupby("host_id")["reviews_per_month"].transform("mean"))

filtered_airbnb["review_scores_rating"] = filtered_airbnb["review_scores_rating"].fillna(filtered_airbnb.groupby("neighbourhood_cleansed")["review_scores_rating"].transform("mean"))
filtered_airbnb["reviews_per_month"] = filtered_airbnb["reviews_per_month"].fillna(filtered_airbnb.groupby("neighbourhood_cleansed")["reviews_per_month"].transform("mean"))

filtered_airbnb["review_scores_rating"] = filtered_airbnb["review_scores_rating"].fillna(filtered_airbnb.groupby("city")["review_scores_rating"].transform("mean"))
filtered_airbnb["reviews_per_month"] = filtered_airbnb["reviews_per_month"].fillna(filtered_airbnb.groupby("city")["reviews_per_month"].transform("mean"))

filtered_airbnb["review_scores_rating"] = filtered_airbnb["review_scores_rating"].fillna(filtered_airbnb.groupby(["state"])["review_scores_rating"].transform("mean"))
filtered_airbnb["reviews_per_month"] = filtered_airbnb["reviews_per_month"].fillna(filtered_airbnb.groupby(["state"])["reviews_per_month"].transform("mean"))

In [ ]:
filtered_airbnb = filtered_airbnb[filtered_airbnb["market"].notna()]
filtered_airbnb = filtered_airbnb[filtered_airbnb["property_type"].notna()]

In [ ]:
filtered_airbnb.shape

In [ ]:
filtered_airbnb.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
#sophia

In [ ]:
filtered_airbnb = filtered_airbnb[filtered_airbnb["host_since"].notna()]

In [ ]:
filtered_airbnb.shape

In [ ]:
filtered_airbnb.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
import re
filtered_airbnb = filtered_airbnb[filtered_airbnb["city"].notna()]
filtered_airbnb = filtered_airbnb[filtered_airbnb.city.str.contains("^[a-zA-Z]")]
filtered_airbnb.shape

In [ ]:
filtered_airbnb.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
# Ziyue 

In [ ]:
new_price = filtered_airbnb.groupby('neighbourhood_cleansed')['price'].apply(lambda x: x.fillna(x.mean()))
filtered_airbnb['price'] = new_price

In [ ]:
filtered_airbnb['price'].isna().sum()

In [ ]:
new_price_city = filtered_airbnb.groupby('market')['price'].apply(lambda x: x.fillna(x.mean()))
filtered_airbnb['price'] = new_price_city

In [ ]:
filtered_airbnb.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
new_bathroom = filtered_airbnb.groupby('property_type')['bathrooms'].apply(lambda x: x.fillna(x.mean()))
filtered_airbnb['bathrooms'] = new_bathroom

In [ ]:
filtered_airbnb['bathrooms'].isna().sum()

In [ ]:
filtered_airbnb = filtered_airbnb[filtered_airbnb["bathrooms"].notna()]

In [ ]:
new_bedroom = filtered_airbnb.groupby('property_type')['bedrooms'].apply(lambda x: x.fillna(x.mean()))
filtered_airbnb['bedrooms'] = new_bedroom

In [ ]:
filtered_airbnb['bedrooms'].isna().sum()

In [ ]:
new_bed = filtered_airbnb.groupby('property_type')['beds'].apply(lambda x: x.fillna(x.mean()))
filtered_airbnb['beds'] = new_bed

In [ ]:
filtered_airbnb['beds'].isna().sum()

In [ ]:
filtered_airbnb.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
filtered_airbnb = filtered_airbnb[filtered_airbnb["accommodates"].notna()]

In [ ]:
#extract features and target

In [ ]:
features = filtered_airbnb.drop(columns=['id',	'name',	'host_id','host_name','weekly_price','monthly_price','price','country_code'])
features.head()

In [ ]:
features.isna().sum()[filtered_airbnb.isna().sum() > 0]

In [ ]:
features.shape

In [ ]:
#feature engineering

In [ ]:
features['latitude']= features['latitude'].astype(float)
features['longitude']= features['longitude'].astype(float)

In [ ]:
list1 = [str(i).split("-") for i in features["host_since"]]
list2 = [(2022-int(i[0])) for i in list1]
features["host_years"] = list2
features = features.drop(columns = ["host_since"])

In [ ]:
amenity_list = features['amenities'].explode().unique()

for newcol in amenity_list:
    features[newcol]= features['amenities'].apply(lambda x: any([newcol in x])).astype(int)

In [ ]:
features.shape #111 new feature

In [ ]:
neigh = pd.get_dummies(features[["neighbourhood_cleansed"]], prefix="neighbourhood_cleansed")
city = pd.get_dummies(features[["city"]], prefix="city")
state = pd.get_dummies(features[["state"]], prefix="state")
market = pd.get_dummies(features[["market"]], prefix="market")
proper = pd.get_dummies(features[["property_type"]], prefix="property_type")
room = pd.get_dummies(features[["room_type"]], prefix="room_type")
cancel = pd.get_dummies(features[["cancellation_policy"]], prefix="cancellation_policy")
features2 = features.drop(columns = ["neighbourhood_cleansed", "city", "state", "market", "property_type", "room_type", "cancellation_policy", "amenities"])
features2 = pd.concat([neigh, city, state, market, proper, room, cancel, features2], axis = 1)
features3 = features.drop(columns = ["neighbourhood_cleansed", "city", "state", "market", "property_type", "room_type", "cancellation_policy", "amenities"])
features3 = pd.concat([market, proper, room, cancel, features3], axis = 1)
features2

In [ ]:
features[['market']]

# New Section

In [ ]:
features.columns

In [ ]:
features.shape

In [ ]:
target = filtered_airbnb[["price"]]
target.head()

In [ ]:
target.isna().sum()[target.isna().sum() > 0]

In [ ]:
#Data Exploration

In [ ]:
#Data Visualization

In [ ]:
map = gpd.read_file('cb_2018_us_state_500k.shp')
map.plot(figsize=(10, 10))
plt.xlim(-180,-60);
plt.ylim(10,75);

In [ ]:
map.head()

In [ ]:
#merge the map dataframe with feature dataframe
merged = map.set_index('STUSPS').merge(filtered_airbnb, how='left', left_on="STUSPS", right_on="state")
merged = merged.fillna(0)

In [ ]:
# set a variable that will call whatever column we want to visualise on the map
variable = 'price'
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 10))
# create map
merged.plot(column=variable, cmap='Reds', linewidth=0.8, ax=ax, legend=True, edgecolor='0.8',figsize=(15, 10))
plt.xlim(-180,-60)
plt.ylim(10,75)
plt.title('Airbnb Listing Price in US')
plt.xlabel('latitude')
plt.ylabel('longtitude');
#fig.savefig("map.png")

In [ ]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
# Create an interactive map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 100:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(filtered_airbnb)):
    Circle(
        location=[float(filtered_airbnb.iloc[i]['latitude']), float(filtered_airbnb.iloc[i]['longitude'])],
        radius=20,
        color=color_producer(filtered_airbnb.iloc[i]['price'])).add_to(m_4)

# Display the map
m_4

In [ ]:
filtered_airbnb.head()

In [ ]:
g = sns.swarmplot(y = "market",
              x = 'price', 
              data = filtered_airbnb,
              # Decrease the size of the points to avoid crowding 
              size = 4)
g.figure.set_size_inches(18,20)
plt.show()

fig = g.get_figure()
fig.savefig("market2.png")

In [ ]:
g = sns.swarmplot(y = "property_type",
              x = 'price', 
              data = filtered_airbnb,
              # Decrease the size of the points to avoid crowding 
              size = 4)
g.figure.set_size_inches(17,20)
plt.show()

fig = g.get_figure()
fig.savefig("property_type.png")

In [ ]:
g = sns.swarmplot(y = "room_type",
              x = 'price', 
              data = filtered_airbnb)
g.figure.set_size_inches(10,7)
plt.show()

fig = g.get_figure()
fig.savefig("room_type.png")

In [ ]:
g = sns.swarmplot(y = "cancellation_policy",
              x = 'price', 
              data = filtered_airbnb,
              # Decrease the size of the points to avoid crowding 
              size = 5)
g.figure.set_size_inches(13,7)
plt.show()

fig = g.get_figure()
fig.savefig("cancellation_policy.png")

In [ ]:
#Heatmap

In [ ]:
features = pd.concat([target['price'], features], axis = 1)

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()
dataplot = sns.heatmap(features[["host_years", "host_total_listings_count", "accommodates", "bathrooms", "bedrooms", "beds","price"]].corr(), cmap="YlGnBu", annot=True)

In [ ]:
dataplot = sns.heatmap(features[["minimum_nights","maximum_nights", "number_of_reviews","review_scores_rating", "reviews_per_month","price"]].corr(), cmap="YlGnBu", annot=True)

In [ ]:
features['market'].unique().sum()

In [ ]:
pair_1 = features[["price", "market","host_years", "host_total_listings_count"]]
sns.pairplot(pair_1, hue = 'market', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             size = 4)

In [ ]:
pair_2 = features[["price", "market", "accommodates", "bathrooms", "bedrooms", "beds", "minimum_nights"]]
sns.pairplot(pair_2, hue = 'market', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             size = 4)

In [ ]:
pair_3 = features[["price", "market", "number_of_reviews","review_scores_rating", "reviews_per_month"]]
sns.pairplot(pair_3, hue = 'market', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             size = 4)

In [ ]:
#drop irrelevant columns

In [ ]:
features3 = features3.drop(columns = ["host_years", "host_total_listings_count", "minimum_nights", "maximum_nights","review_scores_rating", "latitude", "longitude"])
pd.set_option('max_columns', 2000)
features3.head(100)
for i in features3[amenity_list].columns:
  if sum(features3[i]) < 500:
    features3 = features3.drop(columns = [i])


In [ ]:
features3 = features3.drop(columns = ["translation missing: en.hosting_amenity_49", "translation missing: en.hosting_amenity_50"])
features3

In [ ]:
# eliminate outliers

In [ ]:
def outlier_treatment(data):
 sorted(data)
 Q1,Q3 = np.percentile(data , [25,75])
 IQR = Q3-Q1
 lower_range = Q1-(1.5 * IQR)
 upper_range = Q3+(1.5 * IQR)
 return lower_range,upper_range

In [ ]:
combined = pd.concat([target["price"], features3], axis = 1)
lowerbound,upperbound = outlier_treatment(combined["price"])
combined[(combined.price < lowerbound)|(combined.price > upperbound)]

combined.drop(combined[(combined.price < lowerbound)|(combined.price > upperbound)].index , inplace=True)

In [ ]:
plt.hist(combined.price)
plt.xlabel('Count')
plt.ylabel('Prices')

In [ ]:
plt.hist(target.price)
plt.xlabel('Count')
plt.ylabel('Prices')

In [ ]:
target = combined.price
features3 = combined.drop(columns = ["price"])

In [ ]:
#split our data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features3, target, test_size=0.3, random_state=1)

In [ ]:
#Machine Learning Methods

In [ ]:
#Linear Regression (KL)

reg = LinearRegression()
reg.fit(X_train, y_train)

print('Mean Squared Error of training---')
y_pred_train=reg.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=reg.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
# r2 = r2_score(y_train, y_pred_train)
# print('R2 score of training---')
# print(r2)
r2_val_score = cross_val_score(reg, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

# print('---normalized---')

# #normalized value
# reg.fit(normalize(X_train), y_train)
# print('Mean Squared Error of training---')
# y_pred_train=reg.predict(normalize(X_train))
# print(mean_squared_error(y_train, y_pred_train))
# y_pred_test=reg.predict(normalize(X_test))
# print('Mean Squared Error of test---')
# print(mean_squared_error(y_test, y_pred_test))
# # r2 = r2_score(y_train, y_pred_train)
# # print('R2 score of training---')
# # print(r2)
# r2_val_score = cross_val_score(reg, X_train, y_train, cv=5, scoring='r2')
# print('R2 cross validation score of training---')
# print(r2_val_score)
# print('R2 mean cross validation score of training---')
# print(r2_val_score.mean())
# print('R2 score of test---')
# print(r2_score(y_test, y_pred_test))

In [ ]:
#regressors

In [ ]:
bag = BaggingRegressor()
bag.fit(X_train, y_train)
print('Mean Squared Error of training---')
y_pred_train=bag.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=bag.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
# r2 = r2_score(y_train, y_pred_train)
# print('R2 score of training---')
# print(r2)
r2_val_score = cross_val_score(bag, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

# print('---normalized---')

# #normalized value
# bag.fit(normalize(X_train), y_train)
# print('Mean Squared Error of training---')
# y_pred_train=bag.predict(normalize(X_train))
# print(mean_squared_error(y_train, y_pred_train))
# y_pred_test=bag.predict(normalize(X_test))
# print('Mean Squared Error of test---')
# print(mean_squared_error(y_test, y_pred_test))
# # r2 = r2_score(y_train, y_pred_train)
# # print('R2 score of training---')
# # print(r2)
# r2_val_score = cross_val_score(bag, X_train, y_train, cv=5, scoring='r2')
# print('R2 cross validation score of training---')
# print(r2_val_score)
# print('R2 mean cross validation score of training---')
# print(r2_val_score.mean())
# print('R2 score of test---')
# print(r2_score(y_test, y_pred_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Setup the hyperparameter grid
param = {"n_estimators": np.arange(100,500,100)}

# Instantiate the GridSearchCV object: logreg_cv
cv = RandomizedSearchCV(bag, param, cv=5, scoring='r2')

# Fit it to the data
cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Bagging Regressor Parameters: {}".format(cv.best_params_)) 
print("Best score is {}".format(cv.best_score_))

In [ ]:
#optimal bagging
bag = BaggingRegressor(n_estimators=400)

bag.fit(X_train, y_train)
print('Mean Squared Error of training---')
y_pred_train=bag.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=bag.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
r2_val_score = cross_val_score(bag, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

In [ ]:
adaboost = AdaBoostRegressor()
adaboost.fit(X_train, y_train)
print('Mean Squared Error of training---')
y_pred_train=adaboost.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=adaboost.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
# r2 = r2_score(y_train, y_pred_train)
# print('R2 score of training---')
# print(r2)
r2_val_score = cross_val_score(adaboost, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

# print('---normalized---')

# #normalized value
# adaboost.fit(normalize(X_train), y_train)
# print('Mean Squared Error of training---')
# y_pred_train=adaboost.predict(normalize(X_train))
# print(mean_squared_error(y_train, y_pred_train))
# y_pred_test=adaboost.predict(normalize(X_test))
# print('Mean Squared Error of test---')
# print(mean_squared_error(y_test, y_pred_test))
# # r2 = r2_score(y_train, y_pred_train)
# # print('R2 score of training---')
# # print(r2)
# r2_val_score = cross_val_score(adaboost, X_train, y_train, cv=5, scoring='r2')
# print('R2 cross validation score of training---')
# print(r2_val_score)
# print('R2 mean cross validation score of training---')
# print(r2_val_score.mean())
# print('R2 score of test---')
# print(r2_score(y_test, y_pred_test))

In [ ]:
# Setup the hyperparameter grid
param = {"n_estimators": np.arange(100,500,100),
         "loss":["linear", "square", "exponential"]}

# Instantiate the GridSearchCV object: logreg_cv
cv = RandomizedSearchCV(adaboost, param, cv=5, scoring='r2')

# Fit it to the data
cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned AdaBoost Regressor Parameters: {}".format(cv.best_params_)) 
print("Best score is {}".format(cv.best_score_))

In [ ]:
#optimal adaboost
adaboost = AdaBoostRegressor(n_estimators=100, loss='linear')
adaboost.fit(X_train, y_train)
print('Mean Squared Error of training---')
y_pred_train=adaboost.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=adaboost.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
r2_val_score = cross_val_score(adaboost, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

In [ ]:
#Decision Trees Regressor (SJ)
#original value
regressor = DecisionTreeRegressor(random_state = 42, max_depth = 100) 
regressor.fit(X_train, y_train)
print('Mean Squared Error of training---')
y_pred_train=regressor.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=regressor.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
# r2 = r2_score(y_train, y_pred_train)
# print('R2 score of training---')
# print(r2)
r2_val_score = cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

# print('---normalized---')

# #normalized value
# regressor.fit(normalize(X_train), y_train)
# print('Mean Squared Error of training---')
# y_pred_train=regressor.predict(normalize(X_train))
# print(mean_squared_error(y_train, y_pred_train))
# y_pred_test=regressor.predict(normalize(X_test))
# print('Mean Squared Error of test---')
# print(mean_squared_error(y_test, y_pred_test))
# # r2 = r2_score(y_train, y_pred_train)
# # print('R2 score of training---')
# # print(r2)
# r2_val_score = cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2')
# print('R2 cross validation score of training---')
# print(r2_val_score)
# print('R2 mean cross validation score of training---')
# print(r2_val_score.mean())
# print('R2 score of test---')
# print(r2_score(y_test, y_pred_test))

In [ ]:
# Setup the hyperparameter grid
param_dist = {"max_depth": np.arange(1, regressor.tree_.max_depth+1),
              "max_features": np.arange(1,10),
              "min_samples_leaf": np.arange(1, 10)}

# Instantiate the GridSearchCV object: logreg_cv
cv = GridSearchCV(regressor, param_dist, cv=5, scoring='r2')

# Fit it to the data
cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Decision Tree Regression Parameters: {}".format(cv.best_params_)) 
print("Best score is {}".format(cv.best_score_))

In [ ]:
#optimal tree
regressor = DecisionTreeRegressor(max_depth= 9, max_features=8, min_samples_leaf=6) 
regressor.fit(X_train, y_train)
print('Mean Squared Error of training---')
y_pred_train=regressor.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=regressor.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
r2_val_score = cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

In [ ]:
#Random Forest Regressor (JY)
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=5, min_samples_leaf=5, 
                           n_estimators = 500, random_state=88)
rf.fit(X_train, y_train)

print('Mean Squared Error of training---')
y_pred_train=rf.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=rf.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
# r2 = r2_score(y_train, y_pred_train)
# print('R2 score of training---')
# print(r2)
r2_val_score = cross_val_score(rf, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

# print('---normalized---')

# print('Mean Squared Error of training---')
# y_pred_train=rf.predict(normalize(X_train))
# print(mean_squared_error(y_train, y_pred_train))
# y_pred_test=rf.predict(normalize(X_test))
# print('Mean Squared Error of test---')
# print(mean_squared_error(y_test, y_pred_test))
# # r2 = r2_score(y_train, y_pred_train)
# # print('R2 score of training---')
# # print(r2)
# r2_val_score = cross_val_score(rf, X_train, y_train, cv=5, scoring='r2')
# print('R2 cross validation score of training---')
# print(r2_val_score)
# print('R2 mean cross validation score of training---')
# print(r2_val_score.mean())
# print('R2 score of test---')
# print(r2_score(y_test, y_pred_test))

In [ ]:
# Setup the hyperparameter grid
param = {"max_depth": np.arange(1, 32),
              "max_features": np.arange(1,10),
              "min_samples_leaf": np.arange(1, 10),
              "n_estimators": np.arange(100,500,100)}

# Instantiate the GridSearchCV object: logreg_cv
cv = RandomizedSearchCV(rf, param, cv=5, scoring='r2')

# Fit it to the data
cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Random Forest Regression Parameters: {}".format(cv.best_params_)) 
print("Best score is {}".format(cv.best_score_))

In [ ]:
#optimal forest
rf = RandomForestRegressor(n_estimators=300, min_samples_leaf=1, max_features=9, max_depth=19)
rf.fit(X_train, y_train)

print('Mean Squared Error of training---')
y_pred_train=rf.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=rf.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
r2_val_score = cross_val_score(rf, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

In [ ]:
#Neural Network (KL)
from sklearn.neural_network import MLPRegressor
from sklearn import metrics

clf = MLPRegressor(hidden_layer_sizes=(500, 200), max_iter=100, activation='relu', learning_rate_init=0.1,
                   solver = 'adam',  random_state=42, verbose=10)

clf.fit(X_train, y_train)
y_pred_train=clf.predict(X_train)

print('Mean Squared Error of training---')
print(metrics.mean_squared_error(y_train, y_pred_train))
print('R2 on training---')
print(metrics.r2_score(y_train,predictions))

print('---normalized---')

print('Mean Squared Error of training---')
y_pred_train=clf.predict(normalize(X_train))
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=clf.predict(normalize(X_test))

In [ ]:

y_pred_test=clf.predict(X_test)
print('Mean Squared Error of test---')
print(metrics.mean_squared_error(y_test, y_pred_test))
print('R2 on test---')
print(metrics.r2_score(y_test),y_pred_test))

print('---normalized---')

y_pred_test=clf.predict(normalize(X_test))
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))

In [ ]:
# Neural Network with Sequential

In [ ]:
import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

model = Sequential()
model.add(Dense(123, input_dim=109, kernel_initializer='normal', activation='relu'))
model.add(Dense(2670, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=100, batch_size=150, verbose=1, validation_split=0.2)
predictions = model.predict(X_train)
print('R2 on train---')
print(metrics.r2_score(y_train,predictions))

In [ ]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='middle')
plt.show()

In [ ]:
# "MSE"
plt.plot(history.history['mse'])
plt.title('Model MSE')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train'], loc='middle')
plt.show()

In [ ]:
plt.savefig("mse.png")

In [ ]:
# Producing a dataframe of feature importances
ft_weights_xgb_reg = pd.DataFrame(model.feature_importances_, columns=['weight'], index=X_train.columns)
ft_weights_xgb_reg.sort_values('weight', inplace=True)

# Plotting feature importances
plt.figure(figsize=(8,20))
plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
plt.title("Feature importances in the XGBoost model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.show()

In [ ]:
#SVM (JY)
from sklearn import svm
from sklearn.metrics import mean_squared_error
svm = svm.SVR(kernel='rbf')
svm = svm.fit(X_train, y_train)

print('Mean Squared Error of training---')
y_pred_train=svm.predict(X_train)
print(mean_squared_error(y_train, y_pred_train))
y_pred_test=svm.predict(X_test)
print('Mean Squared Error of test---')
print(mean_squared_error(y_test, y_pred_test))
# r2 = r2_score(y_train, y_pred_train)
# print('R2 score of training---')
# print(r2)
r2_val_score = cross_val_score(svm, X_train, y_train, cv=5, scoring='r2')
print('R2 cross validation score of training---')
print(r2_val_score)
print('R2 mean cross validation score of training---')
print(r2_val_score.mean())
print('R2 score of test---')
print(r2_score(y_test, y_pred_test))

# print('---normalized---')

# print('Mean Squared Error of training---')
# y_pred_train=svm.predict(normalize(X_train))
# print(mean_squared_error(y_train, y_pred_train))
# y_pred_test=svm.predict(normalize(X_test))
# print('Mean Squared Error of test---')
# print(mean_squared_error(y_test, y_pred_test))
# # r2 = r2_score(y_train, y_pred_train)
# # print('R2 score of training---')
# # print(r2)
# r2_val_score = cross_val_score(svm, X_train, y_train, cv=5, scoring='r2')
# print('R2 cross validation score of training---')
# print(r2_val_score)
# print('R2 mean cross validation score of training---')
# print(r2_val_score.mean())
# print('R2 score of test---')
# print(r2_score(y_test, y_pred_test))

In [ ]:
# Setup the hyperparameter grid
param = {'C': [0.1,1, 10, 100], 
              'gamma': [1,0.1,0.01,0.001],
              'kernel': ['rbf', 'poly']}

# Instantiate the GridSearchCV object: logreg_cv
cv = RandomizedSearchCV(svm, param, cv=5, scoring='r2')

# Fit it to the data
cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned SVM Parameters: {}".format(cv.best_params_)) 
print("Best score is {}".format(cv.best_score_))